### Upload our custom .jsonl collection of instructions and outputs

In [1]:
import json

file_path = 'test.jsonl'

data = []
with open(file_path, 'r') as file:
    for line in file:
        json_line = json.loads(line.strip())
        data.append(json_line)

print(data[0])

{'instruction': 'What are some of the best museums to visit in Paris?', 'output': 'Paris is a city rich in museums, catering to a wide range of interests. Here are some you might want to consider:\n\n1. The Louvre: The world\'s largest art museum and a historic monument in Paris, home to thousands of works of art, including the Mona Lisa.\n\n2. Musée d\'Orsay: Located in a former railway station, this museum houses a large collection of Impressionist and Post-Impressionist masterpieces.\n\n3. Centre Pompidou: Known for its high-tech architecture, this museum houses the National Museum of Modern Art.\n\n4. Musée de l\'Orangerie: Located in the Tuileries Gardens, it\'s famous for Monet\'s "Water Lilies" series.\n\n5. Musée Rodin: Dedicated to the works of the French sculptor Auguste Rodin.\n\n6. Musée du Quai Branly: Features indigenous art, cultures and civilizations from Africa, Asia, Oceania, and the Americas.\n\n7. Musée Picasso: Home to an impressive collection of more than 5,000 wo

In [2]:
transformed_data = {}
for key in data[0].keys():  # initialize keys
    transformed_data[key] = []

for item in data:
    for key in item:
        transformed_data[key].append(item[key])

In [3]:
from datasets import Dataset
dataset = Dataset.from_dict(transformed_data)
print(transformed_data)

{'instruction': ['What are some of the best museums to visit in Paris?', 'What is the local cuisine like in Paris?', 'What are some popular outdoor activities in Paris?', 'What is the currency used in Paris and where can I exchange my money?', 'What are some of the most famous landmarks in Paris?', 'What is the best time of year to visit Paris?', 'What is the weather like in Paris throughout the year?', 'What are some popular outdoor activities in Paris?', 'What are some popular markets in Paris that I should visit?'], 'output': ['Paris is a city rich in museums, catering to a wide range of interests. Here are some you might want to consider:\n\n1. The Louvre: The world\'s largest art museum and a historic monument in Paris, home to thousands of works of art, including the Mona Lisa.\n\n2. Musée d\'Orsay: Located in a former railway station, this museum houses a large collection of Impressionist and Post-Impressionist masterpieces.\n\n3. Centre Pompidou: Known for its high-tech archite

/home/ubuntu/.conda/envs/genaiplayground/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit", # "unsloth/mistral-7b" for 16bit loading
    max_seq_length = max_seq_length,
    dtype = None, # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth: Fast Mistral patching release 2024.1
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.691 GB
O^O/ \_/ \    CUDA capability = 8.6. Xformers = 0.0.23.post1. FA = False.
\        /    Pytorch version: 2.1.2. CUDA Toolkit = 11.8
 "-____-"     bfloat16 = TRUE. Platform = Linux

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` attribute will be overwritten with the one you passed to `from_pretrained`.


In [6]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN  # add EOS_TOKEN, otherwise your generation will go on forever!
        texts.append(text)
    return { "text" : texts, }
pass

new_dataset = dataset.map(formatting_prompts_func, batched = True,)
print(new_dataset[0])
print(new_dataset[0]['text'])


Map: 100%|██████████| 9/9 [00:00<00:00, 4364.52 examples/s]

{'instruction': 'What are some of the best museums to visit in Paris?', 'output': 'Paris is a city rich in museums, catering to a wide range of interests. Here are some you might want to consider:\n\n1. The Louvre: The world\'s largest art museum and a historic monument in Paris, home to thousands of works of art, including the Mona Lisa.\n\n2. Musée d\'Orsay: Located in a former railway station, this museum houses a large collection of Impressionist and Post-Impressionist masterpieces.\n\n3. Centre Pompidou: Known for its high-tech architecture, this museum houses the National Museum of Modern Art.\n\n4. Musée de l\'Orangerie: Located in the Tuileries Gardens, it\'s famous for Monet\'s "Water Lilies" series.\n\n5. Musée Rodin: Dedicated to the works of the French sculptor Auguste Rodin.\n\n6. Musée du Quai Branly: Features indigenous art, cultures and civilizations from Africa, Asia, Oceania, and the Americas.\n\n7. Musée Picasso: Home to an impressive collection of more than 5,000 wo

In [10]:
# Split the dataset into training and testing sets
train_test_split = new_dataset.train_test_split(test_size=0.1)  # Here, 10% is used for testing

# Access the train and test datasets
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [7]:
# Do model patching and add fast LoRA weights
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length,
)

Unsloth 2024.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [18]:
MAX_STEPS=100
from transformers import TrainingArguments
training_arguments = TrainingArguments(
        output_dir="./unsloth_results",
        evaluation_strategy="steps",
        do_eval=True,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=6,
        per_device_eval_batch_size=4,
        log_level="debug",
        save_steps=100,
        logging_steps=25, 
        learning_rate=2e-4,
        eval_steps=50,
        optim='adamw_8bit',
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        weight_decay=0.1,
        max_steps=MAX_STEPS,
        warmup_ratio=0.01,
        lr_scheduler_type="linear",
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [15]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    tokenizer = tokenizer,
    args = training_arguments,
)


Map: 100%|██████████| 1/1 [00:00<00:00, 564.28 examples/s]
max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend


#### Show current memory stats

In [16]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3090. Max memory = 23.691 GB.
4.625 GB of memory reserved.


In [17]:
trainer_stats = trainer.train()

Currently training with a batch size of: 4
***** Running training *****
  Num examples = 8
  Num Epochs = 500
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 6
  Total optimization steps = 500
  Number of trainable parameters = 41,943,040
Unsloth: `use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`


Step,Training Loss,Validation Loss
50,0.004000,2.322770
100,0.002000,2.475006
150,0.002000,2.430656


***** Running Evaluation *****
  Num examples = 1
  Batch size = 4
***** Running Evaluation *****
  Num examples = 1
  Batch size = 4
Saving model checkpoint to ./unsloth_results/tmp-checkpoint-100
tokenizer config file saved in ./unsloth_results/tmp-checkpoint-100/tokenizer_config.json
Special tokens file saved in ./unsloth_results/tmp-checkpoint-100/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1
  Batch size = 4


KeyboardInterrupt: 

#### Show final memory and time stats

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

#### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
]*1, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

#### Inference
With text streamer

In [ ]:
# alpaca_prompt = Copied from above

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
]*1, return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

#### Saving & loading fine-tuned models
To save the final model as LoRA adapters, either use Huggingface's push_to_hub for an online save or save_pretrained for a local save.

[NOTE] This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("mistral_lora_model") # local saving
# model.push_to_hub("your_name/lora_model", token = "...") # Huggingface hub Online saving

#### Saving to float16 for VLLM

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if True: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

#### GGUF / llama.cpp Conversion
To save to GGUF / llama.cpp, we support it natively now! We clone llama.cpp and we default save it to q8_0. We allow all methods like q4_k_m. Use save_pretrained_gguf for local saving and push_to_hub_gguf for uploading to HF.

In [ ]:
### GGUF / llama.cpp Conversion
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if True: model.save_pretrained_gguf("model_q4_k_m_gguf", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

#### Now, use the model-unsloth.gguf file or model-unsloth-Q4_K_M.gguf file in llama.cpp or a UI based system like GPT4All

In [ ]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, "mistral_lora_model")

In [ ]:
from typing import List

def get_response(query:str, input="")->List[str]:
  inputs = tokenizer(
       [
    alpaca_prompt.format(
        query, # instruction
        input, # input
        "", # output
    )
    ]*1, return_tensors = "pt").to("cuda")
  outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
  return tokenizer.batch_decode(outputs)

query = "Give me some highlights of the following city"
input = "Paris"
resp = get_response(query, input)
def format_msg(message):
    split_msg = message.split("### ")
    final_str = split_msg[1]+split_msg[3]
    return final_str
print(format_msg(resp[0]))